In [1]:
import sys
if '/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/transforms/' not in sys.path:
    sys.path.append('/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/transforms/')
from fanc_seg_utils.database import Neuron_database
import numpy as np
from matplotlib import pyplot as plt
import pymaid
from scipy.spatial import distance 
from concurrent import futures
from matplotlib import pyplot as plt
from scipy import stats
from scipy.stats import norm
import pandas as pd
import marshmallow as mm
from cloudvolume import CloudVolume
from fanc_seg_utils import neuroglancer_utilities
import collections
import six
import json
import meshparty
from meshparty import trimesh_io, trimesh_vtk


In [2]:
db = Neuron_database('/Users/brandon/Documents/Repositories/Python/Neuron_Database/Old/9A.csv',segmentation_version='V4')
db.get_database()

Database active


In [ ]:
db.neurons

In [8]:
db.initialize_catmaid_instance('/Users/brandon/cloudvolume/catmaid_keys.txt',60)

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


In [3]:
sk=db.get_skeletons(73468475091546667)

Shard Indices: 1it [00:00,  5.69it/s]
Minishard Indices: 1it [00:00,  5.59it/s]
Decompressing: 100%|██████████| 1/1 [00:00<00:00, 228.52it/s]


In [9]:
pymaid.upload_neuron(sk,import_annotations=True,remote_instance=db.target_instance)

Fragments will be joined before import. Continue? [Y/N]  y


{'402354067': {'neuron_id': 1178916,
  'skeleton_id': 1178915,
  'node_id_map': {14972: 17761515,
   14969: 17761516,
   14963: 17761517,
   14957: 17761518,
   14951: 17761519,
   14945: 17761520,
   14939: 17761521,
   14933: 17761522,
   14927: 17761523,
   14921: 17761524,
   14915: 17761525,
   14909: 17761526,
   14903: 17761527,
   14897: 17761528,
   14891: 17761529,
   14885: 17761530,
   14879: 17761531,
   14873: 17761532,
   14867: 17761533,
   14861: 17761534,
   14855: 17761535,
   14849: 17761536,
   14843: 17761537,
   14837: 17761538,
   14831: 17761539,
   14825: 17761540,
   14819: 17761541,
   14813: 17761542,
   14807: 17761543,
   14801: 17761544,
   14795: 17761545,
   14789: 17761546,
   14783: 17761547,
   14777: 17761548,
   14771: 17761549,
   14765: 17761550,
   14759: 17761551,
   14753: 17761552,
   14747: 17761553,
   14741: 17761554,
   14735: 17761555,
   14729: 17761556,
   14723: 17761557,
   14717: 17761558,
   14711: 17761559,
   14705: 17761560,
  

In [ ]:
mesh = db.get_mesh(73820112385702160)
sk = db.get_skeletons(73820112385702160)

In [ ]:
cv = CloudVolume('graphene://https://api.zetta.ai/wclee/segmentation/table/vnc1_full_v3align_2_v2')

In [ ]:
db.initialize_catmaid_instance('/Users/brandon/Documents/MN_Analysis/catmaid_keys.txt',60)

In [ ]:
output = neuroglancer_utilities.fanc4_to_3(nodes.values)
neuron.nodes[['x','y','z']] = np.array(list(zip(output['x'],output['y'],output['z']))) * np.array([4.3,4.3,45])

In [ ]:
CloudVolume('graphene://https://wclee.dev.zetta.ai/segmentation/table/vnc1_full_v3align_2_v2', use_https=True, agglomerate=True)

In [ ]:
n=pymaid.get_neurons('annotation:for_avinash',remote_instance=db.target_instance)

In [ ]:
neuron = n[1]

In [ ]:
b = neuron.connectors[['x','y','z']].values / np.array([4.3,4.3,45])

In [ ]:
for i in range(len(b)):
    df.iloc[i,0] = b[i]

In [ ]:
db.upload_to_catmaid(73469574737466912,only_good=False)

In [ ]:
seg_id = 648518346661938249
sk = single_skeletonization.single_skeleton(seg_id,
                                       cv_url=db.segmentations[db.segmentation_version],
                                       soma_coords=db.neurons.loc[db.neurons.Segment_ID == seg_id,'V4_Soma'])

In [ ]:

class NumericField(mm.fields.Int):

    def _jsonschema_type_mapping(self):
        return {
            'type': 'integer',
        }
 
class AnnotationSchema(mm.Schema):
    '''schema with the type of annotation'''
    type = mm.fields.Str(
        required=True,
        description='type of annotation',
        drop_column=True)

    valid = mm.fields.Bool(
        required=False,
        description="is this annotation valid"
    )
    
    
class SpatialPoint(mm.Schema):
    '''a position in the segmented volume '''
    position = mm.fields.List(mm.fields.Int,
                              validate=mm.validate.Length(equal=3),
                              required=True,
                              description='spatial position in voxels of'
                                          'x,y,z of annotation',
                              postgis_geometry='POINTZ')

    @mm.post_load
    def transform_position(self, item):
        if self.context.get('postgis', False):
            item['position'] = "POINTZ({} {} {})".format(item['position'][0],
                                                         item['position'][1],
                                                         item['position'][2])
        return item


class BoundSpatialPoint(SpatialPoint):
    ''' a position in the segmented volume that is associated with an object'''
    supervoxel_id = NumericField(missing=mm.missing,
                                 description="supervoxel id of this point")
    root_id = NumericField(description="root id of the bound point",
                           index=True)

    @mm.post_load
    def convert_point(self, item):
        bsp_fn = self.context.get('bsp_fn', None)
        if bsp_fn is not None:
            bsp_fn(item)
        return item

In [ ]:
a = SpatialPoint.from_dict({'postition':[41854, 153786, 2396]})

In [ ]:
a.postition

In [ ]:
db.segmentations

In [ ]:
db.neurons.loc[['good' in i for i in db.neurons.Annotations],'Segment_ID'].values

In [ ]:
db.plot_mesh(74172093612626737)

In [ ]:
volume = db.get_mesh(74172093612626737)

In [ ]:
volume

In [ ]:
pvol = pymaid.Volume(volume[0].vertices,volume[0].faces,name=db.neurons.loc[db.neurons.Segment_ID==74172093612626737,'Name'])

In [ ]:
db.initialize_catmaid_instance('/Users/brandon/Documents/MN_Analysis/catmaid_keys.txt',60)